In [ ]:
# Setup: Load required packages
options(dplyr.summarise.inform = FALSE)
library(tidyverse)
library(mlba)

---

## Part 1: Data Summaries — Understanding Your Data

### 🏢 Business Context

Before any analysis, you must **know your data**. Summary statistics answer critical questions:

| Stakeholder Question | Statistic | Business Insight |
|---------------------|-----------|------------------|
| "What's typical?" | Mean, Median | Central tendency, expected value |
| "How variable is it?" | SD, Range | Risk, volatility, consistency |
| "Are there extremes?" | Min, Max | Outliers, edge cases |
| "Is data complete?" | Missing count | Data quality issues |

### Example 1: Boston House Prices

We'll analyze the Boston Housing dataset — used by real estate companies, banks, and urban planners.

In [ ]:
boston.housing.df <- mlba::BostonHousing
head(boston.housing.df, 9)

In [ ]:
summary(boston.housing.df)

### 📋 Interpreting `summary()` Output

**For each variable, you see:**
- **Min/Max**: Range of values (detect outliers)
- **1st Quartile (Q1)**: 25% of values are below this
- **Median**: Middle value (50th percentile) — robust to outliers
- **Mean**: Average — sensitive to extreme values
- **3rd Quartile (Q3)**: 75% of values are below this

**Business insight**: If Mean >> Median → right-skewed (few very high values). Common in income, property values.

### Computing Individual Statistics

For detailed analysis of a single variable:

In [ ]:
# compute mean, standard dev., min, max, median, length, and missing values of CRIM
mean(boston.housing.df$CRIM)
sd(boston.housing.df$CRIM)
min(boston.housing.df$CRIM)
max(boston.housing.df$CRIM)
median(boston.housing.df$CRIM)
length(boston.housing.df$CRIM)

# find the number of missing values of variable CRIM
sum(is.na(boston.housing.df$CRIM))

### 📋 Understanding Standard Deviation (SD)

**SD** measures **spread** or **variability**:
- Low SD → Values cluster tightly around mean (consistent, predictable)
- High SD → Values widely dispersed (high variability, risk)

**Business example**: 
- Product quality with SD = 0.1 → Very consistent
- Stock returns with SD = 30% → Very volatile (risky)

### Computing Statistics for All Variables

Create a comprehensive summary table:

In [ ]:
# compute mean, standard dev., min, max, median, length, and missing values for all variables
data.frame(mean=sapply(boston.housing.df, mean),
           sd=sapply(boston.housing.df, sd),
           min=sapply(boston.housing.df, min),
           max=sapply(boston.housing.df, max),
           median=sapply(boston.housing.df, median),
           length=sapply(boston.housing.df, length),
           miss.val=sapply(boston.housing.df,
                           function(x) sum(length(which(is.na(x))))))

### 📋 How to Use This Summary Table

**For each row (variable), check:**
1. **miss.val**: Any missing values? (Needs imputation?)
2. **min/max**: Extreme values? (Outliers? Data errors?)
3. **sd**: High variability? (May dominate model if not scaled)
4. **mean vs. median**: Large difference? (Skewed distribution?)

### Correlation Matrix

Understand relationships between variables:

In [ ]:
round(cor(boston.housing.df),2)

### 📋 Interpreting Correlations

| Correlation | Interpretation | Action |
|-------------|----------------|--------|
| **r ≈ +1.0** | Perfect positive relationship | Variables are redundant |
| **r ≈ +0.7 to +0.9** | Strong positive | Consider removing one (multicollinearity) |
| **r ≈ 0** | No linear relationship | Variables are independent |
| **r ≈ -0.7 to -1.0** | Strong negative | Inverse relationship |

**Business insight**: If two variables correlate at r > 0.9, they measure nearly the same thing. Keep only one to avoid redundancy.

---

## Part 2: Aggregation and Pivot Tables

### 🏢 Business Context: Slicing and Dicing Data

Aggregation answers questions like:
- "What's the average sale price **by region**?"
- "How does customer satisfaction vary **by product and store**?"
- "What are total revenues **by quarter and product line**?"

### Frequency Tables

Count observations in each category:

In [ ]:
table(boston.housing.df$CHAS)

In [ ]:
# tidyverse version
boston.housing.df %>% count(CHAS)

### 📋 Interpreting Frequency Tables

**CHAS** indicates whether the tract bounds the Charles River:
- **0**: Does not bound river (majority)
- **1**: Bounds river (minority)

**Business use**: Check for class imbalance before modeling (remember Module 1.1!).

### Creating Bins for Continuous Variables

Convert continuous data into categories for easier interpretation:

In [ ]:
# create bins of size 1 for number of rooms (RM)
boston.housing.df <- boston.housing.df %>%
  mutate(RM.bin = cut(RM, c(1:9), labels=FALSE))

head(boston.housing.df %>% select(RM, RM.bin))

### Two-Way Aggregation: Pivot Tables

Compute averages across multiple dimensions:

In [ ]:
# compute the average of MEDV by (binned) RM and CHAS
# in aggregate() use the argument by= to define the list of aggregating variables,
# and FUN= as an aggregating function.
aggregate(boston.housing.df$MEDV, by=list(RM=boston.housing.df$RM.bin,
          CHAS=boston.housing.df$CHAS), FUN=mean)

In [ ]:
# tidyverse version (cleaner syntax)
boston.housing.df %>%
  group_by(RM.bin, CHAS) %>%
  summarise(mean(MEDV))

### 📋 Interpreting Pivot Tables

**What this shows**:
- Each row = combination of RM.bin and CHAS
- `mean(MEDV)` = average median home value for that group

**Business insight**: 
- More rooms → Higher median value (as expected)
- River proximity (CHAS=1) → Premium pricing

### Creating Cross-Tabulated Pivot Tables

Use `reshape` package for traditional Excel-style pivot tables:

In [ ]:
library(reshape)
boston.housing.df <- mlba::BostonHousing
# create bins of size 1
boston.housing.df <- boston.housing.df %>%
  mutate(RM.bin = cut(RM, c(1:9), labels=FALSE))

# use melt() to stack a set of columns into a single column of data.
# stack MEDV values for each combination of (binned) RM and CHAS
mlt <- melt(boston.housing.df, id=c("RM.bin", "CHAS"), measure=c("MEDV"))
head(mlt, 5)

In [ ]:
# use cast() to reshape data and generate pivot table
cast(mlt, RM.bin ~ CHAS, subset=variable=="MEDV",
     margins=c("grand_row", "grand_col"), mean)

In [ ]:
# tidyverse version (simpler and more readable)
boston.housing.df %>%
  group_by(RM.bin, CHAS) %>%
  summarize(mean=mean(MEDV)) %>%
  spread(CHAS, mean)

### 📋 Reading Pivot Tables

**Structure**:
- **Rows**: RM.bin (number of rooms category)
- **Columns**: CHAS (0 = no river, 1 = river)
- **Values**: Average MEDV (median home value in $1000s)

**Business application**: This is exactly how you'd analyze:
- Sales by Region × Product Category
- Customer satisfaction by Store × Service Type
- Revenue by Quarter × Sales Channel

---

## Part 3: Reducing Categories in Categorical Variables

### 🏢 Business Context: Simplification for Clarity

Sometimes categorical variables have **too many levels**:
- 50 states → Regions (Northeast, South, Midwest, West)
- 100 product SKUs → Product families
- 20 age groups → Young/Middle/Senior

**Benefits**:
- Easier interpretation for stakeholders
- More stable models (avoid overfitting)
- Clearer visualizations

### Visualizing Category Distributions

In [ ]:
boston.housing.df <- mlba::BostonHousing

tbl <- table(boston.housing.df$CAT.MEDV, boston.housing.df$ZN)
prop.tbl <- prop.table(tbl, margin=2)
barplot(prop.tbl, xlab="ZN", ylab="", yaxt="n",main="Distribution of CAT.MEDV by ZN")
axis(2, at=(seq(0,1, 0.2)), paste(seq(0,100,20), "%"))

In [ ]:
# ggplot2 version (publication quality)
library(tidyverse)
df <- data.frame(prop.tbl)
ggplot(df, aes(x=Var2, y=Freq, group=Var1, fill=Var1)) +
  geom_bar(stat="identity", color="grey", width=1) +
  scale_y_continuous(labels = scales::percent, expand=expansion()) +
  scale_fill_manual("CAT.MEDV", values=c("0"="#eeeeee", "1"="darkgrey")) +
  labs(x="ZN", y="", title="Distribution of CAT.MEDV by ZN")

### 📋 Interpreting Stacked Bar Charts

**What you see**:
- Each bar = 100% (one ZN category)
- Dark section = proportion with CAT.MEDV = 1 (high value)
- Light section = proportion with CAT.MEDV = 0 (low value)

**Business insight**: If ZN has many categories with similar distributions, consider collapsing them.

### Time Series Example

In [ ]:
library(forecast)
tru.data <- mlba::ToysRUsRevenues
tru.ts <- ts(tru.data[, 3], start = c(1992, 1), end = c(1995, 4), freq = 4)
autoplot(tru.ts) +
  geom_point(size=0.5) +
  labs(x="Time", y="Revenue ($ millions)") +
  theme_bw()

### 📋 Time Series Visualization

**Patterns to look for**:
- **Trend**: Overall upward/downward movement
- **Seasonality**: Regular patterns (Q4 spikes for retail)
- **Outliers**: Unusual points (promotions, disruptions)

**Business application**: Forecast future revenues, plan inventory.

---

## Part 4: Principal Components Analysis (PCA)

### 🏢 Business Context: The Curse of Dimensionality

**Problem**: Your customer dataset has 100 variables. Which ones matter?

**PCA Solution**: Reduces 100 correlated variables to 5-10 **principal components** that capture most of the variation.

### How PCA Works

```
ORIGINAL DATA                    PCA TRANSFORMATION
┌─────────────────┐             ┌─────────────────┐
│ 50 correlated   │     →       │ 5 uncorrelated  │
│ variables       │             │ components      │
│ (multicollinear)│             │ (independent)   │
└─────────────────┘             └─────────────────┘
      Hard to                      Easy to
    interpret                     interpret
```

**Key benefits**:
1. **Dimension reduction**: 50 → 5 variables
2. **Removes multicollinearity**: Components are orthogonal (uncorrelated)
3. **Noise reduction**: Minor components capture noise, not signal

### Example 2: Breakfast Cereals

Start with a simple 2-variable example:

In [ ]:
library(tidyverse)
cereals.df <- mlba::Cereals %>% select(calories, rating)
# compute PCs on two dimensions
pcs <- prcomp(cereals.df %>% select(calories, rating))
summary(pcs)

### 📋 Interpreting PCA Summary

**Key metrics**:
- **Standard deviation**: Spread of data along each component
- **Proportion of Variance**: % of total variability explained
- **Cumulative Proportion**: Running total of variance explained

**Example interpretation**:
- PC1 explains 96% of variance → Captures almost all information
- PC2 explains 4% → Minor patterns, possibly noise

### Principal Component Loadings

In [ ]:
pcs$rotation

### 📋 Understanding Loadings

**Loadings** show how original variables combine to form components:

```
PC1 = (0.685 × calories) + (0.729 × rating)
PC2 = (0.729 × calories) + (-0.685 × rating)
```

**Interpretation**:
- **High positive loading**: Variable strongly contributes in same direction
- **High negative loading**: Variable contributes in opposite direction
- **Near-zero loading**: Variable doesn't contribute to this component

### Component Scores

Each observation gets a score on each component:

In [ ]:
scores <- pcs$x
head(scores, 5)

### 📋 Using Component Scores

**What scores mean**:
- Each cereal has a PC1 score and PC2 score
- Use these scores as new features in downstream models
- Scores are **uncorrelated** → no multicollinearity!

### Visualizing Principal Components

In [ ]:
getPCaxis <- function(f, pcs, pcLabel) {
  return (data.frame(
    rbind(pcs$center + f * pcs$rotation[, pcLabel],
          pcs$center - f * pcs$rotation[, pcLabel]))
  )
}
PC1 <- getPCaxis(90, pcs, "PC1")
PC2 <- getPCaxis(50, pcs, "PC2")
ggplot(cereals.df, aes(x=calories, y=rating)) +
  geom_point() +
  geom_line(data=PC1) +
  geom_line(data=PC2) +
  coord_cartesian(xlim=c(0, 200), ylim=c(0, 110)) +
  labs(x="Calories", y="Rating") +
  annotate(geom="text", x=30, y=80, label="z[1]",parse=TRUE) +
  annotate(geom="text", x=120, y=80, label="z[2]",parse=TRUE) +
  theme_bw()

### 📋 Reading the PCA Plot

**What you see**:
- **Points**: Individual cereals
- **Lines**: Principal component directions
- **z₁ (PC1)**: Direction of maximum variance
- **z₂ (PC2)**: Orthogonal direction (90° to PC1)

**Business insight**: PC1 captures the main pattern, PC2 captures residual variation.

---

### Full PCA on All Cereal Variables

In [ ]:
# load and preprocess the data
cereals.df <- mlba::Cereals %>%
  column_to_rownames("name") %>%
  select(-c(mfr, type)) %>%
  drop_na()

pcs <- prcomp(cereals.df)
summary(pcs)

In [ ]:
pcs$rotation[,1:5]

### 📋 Interpreting Multi-Variable PCA

**How many components to keep?**

| Rule | Criterion |
|------|----------|
| **Cumulative variance** | Keep components explaining 80-90% of variance |
| **Scree plot** | Keep components before the "elbow" |
| **Eigenvalue > 1** | Kaiser criterion (for standardized data) |

**Business decision**: More components = more accuracy, but less interpretability.

### ⚠️ Normalizing Data for PCA

**Critical**: If variables have different scales, PCA will be dominated by large-scale variables!

Example:
- Calories (range: 50-150)
- Sodium (range: 0-300)
- Protein (range: 1-6)

Sodium will dominate PC1 just because of its scale, not its importance.

**Solution**: Use `scale. = TRUE` to standardize all variables:

In [ ]:
# Use function prcomp() with scale. = T to run PCA on normalized data
pcs.cor <- prcomp(cereals.df, scale. = T)

summary(pcs.cor)

In [ ]:
pcs.cor$rotation[,1:5]

### 📋 Normalized vs. Non-Normalized PCA

**When to normalize:**
- Variables on different scales (always normalize!)
- You care about **correlations**, not raw variances

**When NOT to normalize:**
- All variables in same units (e.g., all are percentages)
- Scale differences are meaningful (e.g., larger variance = more important)

**Best practice**: Almost always use `scale. = TRUE` in business applications.

### Visualizing Principal Components

In [ ]:
library(ggrepel)
ggplot(data.frame(pcs.cor$x), aes(x=PC1, y=PC2, label=rownames(pcs.cor$x))) +
  geom_point(shape=21) +
  geom_text_repel(size=2, max.overlaps=7) +
  theme_bw()

### 📋 Interpreting the PCA Biplot

**What you see**:
- Each **point** = one cereal
- **X-axis (PC1)**: First principal component (highest variance)
- **Y-axis (PC2)**: Second principal component
- **Proximity**: Cereals close together are similar

**Business applications**:
- **Market segmentation**: Clusters of similar products
- **Outlier detection**: Points far from center
- **Competitive analysis**: Which products compete directly?

---

## Part 5: Using PCA for Downstream Modeling

### 🏢 Business Context: PCA as Preprocessing

**Workflow**:
1. Run PCA to reduce 50 variables to 10 components
2. Use component scores as features in classification/regression
3. Benefit from uncorrelated, de-noised features

### Example: Wine Classification

In [ ]:
wine.df <- mlba::Wine %>% select(-Type)
pcs.cor <- prcomp(wine.df, scale. = TRUE)
summary(pcs.cor)

In [ ]:
pcs.cor$rotation[,1:4]

### 📋 Deciding How Many Components to Use

**Example decision**:
- Original: 13 variables
- PC1-PC4 explain 73% of variance
- PC1-PC6 explain 85% of variance

**Trade-off**:
- Fewer components → Simpler model, less overfitting, easier interpretation
- More components → Better accuracy, captures more patterns

**Common practice**: Start with components explaining 80-85% of variance.

### 🎯 Next Steps: Classification/Regression

After PCA, you would:
1. Extract component scores: `pcs.cor$x`
2. Use them in models: `lm()`, `glm()`, `randomForest()`, etc.
3. Evaluate performance on holdout set

---

## Summary: Key Takeaways

### 🔧 Essential Functions Reference

#### Data Summaries
| Task | Function | Output |
|------|----------|--------|
| All statistics | `summary(df)` | Min, Q1, median, mean, Q3, max |
| Mean | `mean(x)` | Average value |
| Standard deviation | `sd(x)` | Measure of spread |
| Correlation matrix | `cor(df)` | All pairwise correlations |

#### Aggregation
| Task | Function | Business Use |
|------|----------|-------------|
| Frequency table | `table(x)` | Count by category |
| Group summaries | `group_by() %>% summarise()` | Averages by segment |
| Pivot table | `spread()` or `cast()` | Cross-tabulation |

#### Principal Component Analysis
| Task | Function | Notes |
|------|----------|-------|
| Run PCA | `prcomp(df, scale.=TRUE)` | Always normalize! |
| Summary | `summary(pcs)` | Variance explained |
| Loadings | `pcs$rotation` | Variable contributions |
| Scores | `pcs$x` | New feature values |

---

### 🎯 Best Practices Checklist

✅ **Always check for missing values** before analysis
✅ **Compute correlations** to detect multicollinearity
✅ **Normalize data** (`scale.=TRUE`) before PCA unless all variables are on same scale
✅ **Check variance explained** to decide how many components to keep
✅ **Document your decisions** (why 5 components? why normalize?)
✅ **Visualize results** for stakeholders (biplots, scree plots)

---

### 🏢 Business Value Summary

| Technique | Business Problem | Value |
|-----------|------------------|-------|
| **Summary statistics** | "What does our data look like?" | Quick insights, data quality checks |
| **Pivot tables** | "How do metrics vary by segment?" | Targeted strategies, resource allocation |
| **PCA** | "Which variables really matter?" | Simplified models, reduced storage costs |
| **Dimension reduction** | "Too many features for model" | Prevent overfitting, faster computation |

---

### 📚 Connection to Other Modules

- **Module 1.1**: Use PCA components as features in regression
- **Module 2.1**: Test differences between PCA-derived groups (ANOVA)
- **Module 3**: Advanced dimension reduction (t-SNE, UMAP)
- **Module 4**: Factor Analysis (similar to PCA but different assumptions)
- **Module 5**: Cluster on PCA scores instead of raw variables

---

**Next Steps**: Apply PCA to your own high-dimensional datasets. Start with `scale.=TRUE` and keep components explaining 80-90% of variance!